In [189]:
# Advent of Code 2022
# https://adventofcode.com/2022/day/7

In [190]:
with open("input.txt") as f:
    input = f.read()

    lines = input.split('\n')
    # remove blank lines
    lines = list(filter(None, lines))




In [191]:
##########
# Strategy for creating a directory tree:
##########
 
# Directory tree is a dictionary of dictionaries.  
# Each item in the dictionary is either a file or a directory.
# Format for each item is as follows:
# {<id>:{
#   'item_id':<id>, 
#   'item':<'dir or 'file'>, 
#   'parent_id':<id of the parent directory>,
#   'name':<name of the file or directory>,
#   'contents':[<list of ids for the contents of this directory if the item is a directory>],
#   'size':<file size if the item is a file>,
#   'depth': <how many levels down from the root is this item?>}
# }

# This structure lets you trace any item in the tree to its parent directory and its contents.


In [217]:

def reset_tree():

    tree = {}

    global wd
    global next_id

    '''set up the tree with a root directory,
    # id = 1, parent directory id = 0 (this is an arbitrary id since the root 
    # directory has no actual parent directory.)'''
    tree = {0:{
        'item_id':0, 
        'item':'dir', 
        'parent_id':0, 
        'name':'root', 
        'contents':[], 
        'size':0, 
        'depth':0},
        1:{
        'item_id':1, 
        'item':'dir', 
        'parent_id':0, 
        'name':'root', 
        'contents':[], 
        'size':0, 
        'depth':0}}

    # set root as the default item to start at
    wd = tree[1]

    # set next available ID to assign
    next_id = 2


In [218]:


def cd(l):
    ''' reads in an instruction line (l) and executes CD instructions: 
    either moves to the specified directory or creates it if it doesn't
    exist yet and then moves to it.'''

    global wd
    global next_id
    
    # cd \  -- go to root directory
    if l[5:] == '\\':
        
        # working directory = root directory
        wd = tree[1]

    # cd ..  -- move up one directory level
    elif l[5:] == '..':
        # change working directory to parent directory
        wd = tree[wd['parent_id']]
    
    # cd <directry name>  -- move down one level to specified subdirectory
    else:             
        subdir = l[5:]
        # loop through the contents of the working directory
        # to see if this subdirectory already exists.
        
        # Create a list of directory names under the wd
        subdir_list = []
        for c in wd['contents']:
            subdir_list.append(tree[c]['name'])

        for s in subdir_list:
            # if subdirectory exists, move to it.
            if s == subdir:
                wd = tree[c]
                

        # if subdirectory doesn't exist:
        if subdir not in subdir_list:
            # create subdirectory
            tree[next_id]={
                'item_id':next_id, 
                'item':'dir', 
                'parent_id':wd['item_id'], 
                'name':subdir, 
                'contents':[], 
                'size':0, 
                'depth':wd['depth']+1}
            # add subdirectory to list of items in working directory
            wd['contents'].append(next_id)
            # move to subdirectory
            wd = tree[next_id]
            # update next available id since this one is taken now
            next_id += 1
            



In [219]:

# test the cd function using family tree (easy to follow) and print results. 

test_lines = [
    '$ cd mom', 
    '$ cd me',
    '$ cd ..', 
    '$ cd matt', 
    '$ cd abby', 
    '$ cd ..', 
    '$ cd kermit', 
    '$ cd ..', 
    '$ cd ..', 
    '$ cd amy', 
    '$ cd stets']

for l in test_lines:
    cd(l)

def test_print():
    for i in tree:
        if i == 1:
            pass
        else:
            print(f"name':{tree[i]['name']}")
            print(f"size':{tree[i]['size']}")
            pid = tree[i]['parent_id']
            cid = tree[i]['contents']
            print(f"depth: {tree[i]['depth']}")
            print(f"parent: {tree[pid]['name']}")
            print(f"contents: {[tree[c]['name'] for c in cid]}")
            print('\n')
        
test_print()

name':mom
size':0
depth: 1
parent: root
contents: ['me', 'matt', 'amy']


name':me
size':0
depth: 2
parent: mom
contents: []


name':matt
size':0
depth: 2
parent: mom
contents: ['abby', 'kermit']


name':abby
size':0
depth: 3
parent: matt
contents: []


name':kermit
size':0
depth: 3
parent: matt
contents: []


name':amy
size':0
depth: 2
parent: mom
contents: ['stets']


name':stets
size':0
depth: 3
parent: amy
contents: []


name':nzmddp
size':0
depth: 2
parent: mom
contents: []


name':zjvncc
size':0
depth: 2
parent: mom
contents: ['fwmbbvj', 'hclnfzgv.gqb', 'mpsthvvc', 'qgzdlv.vdh', 'znzszz', 'zwmp', 'bmsw', 'cfgg.btw', 'qfzr.wmv', 'cfgg', 'dppgmmgh', 'hdmc', 'nntzqnfb', 'sfpqf', 'gzhrbrr', 'hbv', 'twf', 'vptsc', 'dcv', 'ddbr.zbq', 'dht', 'hsn', 'sfpqf.cpt', 'sth', 'ttqqpqn', 'twgmlhtp', 'cfmqw', 'dpj', 'dcbntj', 'fpnrzl', 'jrzcqs', 'lbd', 'mbqjsfj', 'nhnnmhj', 'pwjwc', 'pzptv', 'qjhq.brj', 'dswznwtf', 'nzmddp.pln', 'qsq', 'rzbnmn', 'vbr']


name':fwmbbvj
size':0
depth: 3
parent: zjv

In [220]:
##########
# Strategy for executing LS 
##########

# no action needed. LS just indicates the lines that follow
# will require action (adding them to the tree if they don't)
# already exist.

In [221]:
##########
# Strategy for acting on listed directories
##########

# check to see whether directory already exists. If it doesn't, then create it.

In [222]:
# signiicant overlap with cd function above, perhaps 
# break this down into smaller functions later to use in
# both cases and remove duplication

def ls_dir(l):
    '''Reads in an instruction line (l) and treats it
    as a directory listed under an $ ls command. Creates 
    the directory if it doesn't exist yet.'''

    global wd
    global next_id   
            
    subdir = l[4:]
    # loop through the contents of the working directory
    # to see if this subdirectory already exists.
    
    # Create a list of item names under the wd
    item_list = []
    for c in wd['contents']:
        item_list.append(tree[c]['name'])


    # if subdirectory doesn't exist:
    if subdir not in item_list:
        # create subdirectory
        tree[next_id]={
            'item_id':next_id, 
            'item':'dir', 
            'parent_id':wd['item_id'], 
            'name':subdir, 
            'contents':[], 
            'size':0, 
            'depth':wd['depth']+1}
        # add subdirectory to list of items in working directory
        wd['contents'].append(next_id)
        # move to subdirectory
        # wd = tree[next_id]
        # update next available id since this one is taken now
        next_id += 1

In [223]:
# test the ls_dir function

test_lines = [
    'dir new1',
    'dir new2',
    'dir new3',
    'dir new3',
    'dir new2']

for l in test_lines:
    ls_dir(l)

test_print()


name':mom
size':0
depth: 1
parent: root
contents: ['me', 'matt', 'amy']


name':me
size':0
depth: 2
parent: mom
contents: []


name':matt
size':0
depth: 2
parent: mom
contents: ['abby', 'kermit']


name':abby
size':0
depth: 3
parent: matt
contents: []


name':kermit
size':0
depth: 3
parent: matt
contents: []


name':amy
size':0
depth: 2
parent: mom
contents: ['stets']


name':stets
size':0
depth: 3
parent: amy
contents: ['new1', 'new2', 'new3']


name':new1
size':0
depth: 4
parent: stets
contents: []


name':new2
size':0
depth: 4
parent: stets
contents: []


name':new3
size':0
depth: 4
parent: stets
contents: []


name':hclnfzgv.gqb
size':286838
depth: 3
parent: new2
contents: []


name':mpsthvvc
size':0
depth: 3
parent: new2
contents: []


name':qgzdlv.vdh
size':76013
depth: 3
parent: new2
contents: []


name':znzszz
size':186898
depth: 3
parent: new2
contents: []


name':zwmp
size':0
depth: 3
parent: new2
contents: ['dhdgrhg', 'fmdbzs', 'hfczrwl', 'hzdt', 'lchzdqv', 'llfhrcjr', 'mqb'

In [224]:
##########
# Strategy for acting on listed files
##########

# Similar to listed directories - add them if they don't already exist. 
# Also contains overlapping code with other functions
# that might be improved on later.

def ls_file(l):

    global wd
    global next_id 
    
    '''Reads in an instruction line (l) and treats it
    as a file listed under an $ ls command. Creates 
    the file if it doesn't exist yet.'''

    size, name = l.split(' ')
    size = int(size)


    # See if the file already exists.  If not, add it.

    # Create a list of item names under the wd

    item_list = []
    for c in wd['contents']:
        item_list.append(tree[c]['name'])


    # if subdirectory doesn't exist:
    if name not in item_list:
        # create subdirectory
        tree[next_id]={
            'item_id':next_id, 
            'item':'file', 
            'parent_id':wd['item_id'], 
            'name':name, 
            'contents':[], 
            'size':size, 
            'depth':wd['depth']+1}
        # add subdirectory to list of items in working directory
        wd['contents'].append(next_id)
        # move to subdirectory
        # wd = tree[next_id]
        # update next available id since this one is taken now
        next_id += 1



In [225]:
# Test the ls_file function

test_lines = [
    '5068723 test_file_1',
    '4 test2.abc']

for l in test_lines:
    ls_file(l)

test_print()


name':mom
size':0
depth: 1
parent: root
contents: ['me', 'matt', 'amy']


name':me
size':0
depth: 2
parent: mom
contents: []


name':matt
size':0
depth: 2
parent: mom
contents: ['abby', 'kermit']


name':abby
size':0
depth: 3
parent: matt
contents: []


name':kermit
size':0
depth: 3
parent: matt
contents: []


name':amy
size':0
depth: 2
parent: mom
contents: ['stets']


name':stets
size':0
depth: 3
parent: amy
contents: ['new1', 'new2', 'new3', 'test_file_1', 'test2.abc']


name':new1
size':0
depth: 4
parent: stets
contents: []


name':new2
size':0
depth: 4
parent: stets
contents: []


name':new3
size':0
depth: 4
parent: stets
contents: []


name':test_file_1
size':5068723
depth: 4
parent: stets
contents: []


name':test2.abc
size':4
depth: 4
parent: stets
contents: []


name':qgzdlv.vdh
size':76013
depth: 3
parent: new2
contents: []


name':znzszz
size':186898
depth: 3
parent: new2
contents: []


name':zwmp
size':0
depth: 3
parent: new2
contents: ['dhdgrhg', 'fmdbzs', 'hfczrwl', 'hzdt

In [226]:

##########
# Read in the instruction lines and run code
##########

reset_tree()


for l in lines:

    if l[:5] == r'$ cd ':
        cd(l)

    elif l == r'$ ls':
        pass
        

    elif l[:4] == 'dir ':
        ls_dir(l)

    else:
        ls_file(l)





In [227]:
reset_tree()
test_print()

name':/
size':0
depth: 1
parent: root
contents: ['bzcg', 'hrtvrp', 'jvj', 'ltrqb', 'msqlnht', 'mvs', 'nzmddp', 'zjvncc']


name':bzcg
size':0
depth: 2
parent: /
contents: []


name':hrtvrp
size':0
depth: 2
parent: /
contents: []


name':jvj
size':0
depth: 2
parent: /
contents: []


name':ltrqb
size':0
depth: 2
parent: /
contents: []


name':msqlnht
size':0
depth: 2
parent: /
contents: []


name':mvs
size':0
depth: 2
parent: /
contents: []


name':nzmddp
size':0
depth: 2
parent: /
contents: []


name':zjvncc
size':0
depth: 2
parent: /
contents: ['fwmbbvj', 'hclnfzgv.gqb', 'mpsthvvc', 'qgzdlv.vdh', 'znzszz', 'zwmp', 'bmsw', 'cfgg.btw', 'qfzr.wmv', 'cfgg', 'dppgmmgh', 'hdmc', 'nntzqnfb', 'sfpqf', 'gzhrbrr', 'hbv', 'twf', 'vptsc', 'dcv', 'ddbr.zbq', 'dht', 'hsn', 'sfpqf.cpt', 'sth', 'ttqqpqn', 'twgmlhtp', 'cfmqw', 'dpj', 'dcbntj', 'fpnrzl', 'jrzcqs', 'lbd', 'mbqjsfj', 'nhnnmhj', 'pwjwc', 'pzptv', 'qjhq.brj', 'dswznwtf', 'nzmddp.pln', 'qsq', 'rzbnmn', 'vbr']


name':fwmbbvj
size':0
depth: 3

In [246]:
# How deep is the entire tree?
all_depths = [tree[i]['depth'] for i in tree]
max_depth = max(all_depths)

11